In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.cross_validation import train_test_split
from tqdm import tqdm
import gc
import time
from sklearn.preprocessing import scale
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print('loading files...')
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
sample_submission = pd.read_csv('sample_submission.csv')

loading files...


In [3]:
train.head()

,connection_id,cont_1,cont_2,cont_3,cont_4,cont_5,cont_6,cont_7,cont_8,cont_9,...,cat_15,cat_16,cat_17,cat_18,cat_19,cat_20,cat_21,cat_22,cat_23,target
0,cxcon_1,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,2
1,cxcon_4,0,520,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
2,cxcon_7,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
3,cxcon_10,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,0
4,cxcon_13,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,511,511,255,255,2


In [4]:
test.head()

,connection_id,cont_1,cont_2,cont_3,cont_4,cont_5,cont_6,cont_7,cont_8,cont_9,...,cat_14,cat_15,cat_16,cat_17,cat_18,cat_19,cat_20,cat_21,cat_22,cat_23
0,cxcon_2,0,331,6085,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,13,13,255,255
1,cxcon_5,0,520,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,511,511,255,255
2,cxcon_8,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,511,511,255,255
3,cxcon_11,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,510,509,255,255
4,cxcon_14,0,1032,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,511,511,255,255


In [5]:
sample_submission.head()

,connection_id,target
0,cxcon_2,2
1,cxcon_5,0
2,cxcon_8,0
3,cxcon_11,1
4,cxcon_14,2


In [6]:
target = train['target']
test_ID = test['connection_id']
train.drop(['target','connection_id'] , axis=1 , inplace = True)
test.drop(['connection_id'] , axis=1 , inplace = True)
tr_ID = train.shape[0]

data = pd.concat([train, test] , axis = 0)

In [7]:
cont = data.columns[data.columns.str.startswith('cont')]
cat = data.columns[data.columns.str.startswith('cat')]

In [8]:
for col in tqdm(cat) :
    data[col] = data[col].astype('category')
    data[col] = data[col].cat.codes

100%|██████████| 23/23 [00:00<00:00, 35.77it/s]


In [9]:
poly = PolynomialFeatures(degree = 2 ,interaction_only = True)
feat_new = poly.fit_transform(data[cont].values)
data = np.concatenate([data.values , feat_new] , axis = 1)
data = pd.DataFrame(data)

In [10]:
data['n0'] = (data==0).sum(axis=1)
data['n1'] = (data==1).sum(axis=1)
data['n2'] = (data==2).sum(axis=1)
data['n3'] = (data==3).sum(axis=1)
data['n4'] = (data==4).sum(axis=1)
data['n5'] = (data==5).sum(axis=1)
data['n6'] = (data==6).sum(axis=1)

In [11]:
train = data.iloc[:tr_ID]
test = data.iloc[tr_ID:]

In [12]:
# lgb

t1 = time.time()

X = train
X = X.values
y = target.values
sub = np.zeros([test.shape[0],3])


nrounds = 1500
kfold = 5

params = {'metric':'multi_logloss', 'learning_rate' : 0.01, 'max_depth':10, 'max_bin':10,  'objective': 'multiclass', 
          'num_class':3, 'feature_fraction': 0.8,'bagging_fraction':0.9,'bagging_freq':10,  'min_data': 500}

#X_train ,X_eval ,y_train, y_eval = train_test_split(X,y,train_size = 0.8 , random_state = 1)

X_train ,X_eval ,y_train, y_eval = X,X,y,y
lgb_model = lgb.train(params, lgb.Dataset(X_train, label=y_train), nrounds, lgb.Dataset(X_eval, label=y_eval), verbose_eval=10, early_stopping_rounds = 100 )
   
s = lgb_model.predict(test.values, num_iteration=lgb_model.best_iteration)
    
feat_imp = lgb_model.feature_importance()
    
t2 = time.time()
print "Time taken {} hr".format((t2-t1)/(60*60))

Training until validation scores don't improve for 100 rounds.
[10]	valid_0's multi_logloss: 1.02766
[20]	valid_0's multi_logloss: 0.967177
[30]	valid_0's multi_logloss: 0.915144
[40]	valid_0's multi_logloss: 0.870068
[50]	valid_0's multi_logloss: 0.830794
[60]	valid_0's multi_logloss: 0.79639
[70]	valid_0's multi_logloss: 0.766156
[80]	valid_0's multi_logloss: 0.739518
[90]	valid_0's multi_logloss: 0.71585
[100]	valid_0's multi_logloss: 0.694766
[110]	valid_0's multi_logloss: 0.676007
[120]	valid_0's multi_logloss: 0.659177
[130]	valid_0's multi_logloss: 0.644088
[140]	valid_0's multi_logloss: 0.630549
[150]	valid_0's multi_logloss: 0.618315
[160]	valid_0's multi_logloss: 0.607199
[170]	valid_0's multi_logloss: 0.597142
[180]	valid_0's multi_logloss: 0.588009
[190]	valid_0's multi_logloss: 0.579718
[200]	valid_0's multi_logloss: 0.572156
[210]	valid_0's multi_logloss: 0.565227
[220]	valid_0's multi_logloss: 0.558876
[230]	valid_0's multi_logloss: 0.553081
[240]	valid_0's multi_logloss

In [13]:
submission=test_ID.to_frame()
sub = np.zeros([test.shape[0]])
for j in range(test.shape[0]) :
    sub[j] = s[j].argmax()
submission['target'] = sub 

In [14]:
submission.to_csv('lgb_13.csv', index=False, float_format='%.5f')